# Imports

In [ ]:
import PySimpleGUI as sg
import cv2
import numpy as np
import os

# Encryption And Decryption Logic

In [ ]:
def message_to_bin(message):
    """Convert a string message to binary."""
    return ''.join(format(ord(char), '08b') for char in message)


def bin_to_message(binary):
    """Convert binary data to string message."""
    message = ''
    for i in range(0, len(binary), 8):
        byte = binary[i:i+8]
        message += chr(int(byte, 2))
    return message


def encrypt(password, text_path, image_path, output_path):
    output_path += '/'

    # Read the message from the text file
    with open(text_path, 'r', encoding='utf-8') as file:
        message = file.read()

    # Convert the message and password to binary
    message_bin = message_to_bin(message)
    password_bin = message_to_bin(password)

    # Add a unique delimiter between password and message
    # Use a binary representation that doesn't correspond to a printable character
    delimiter = '10101010'

    # Append the password binary, delimiter, and message binary
    data_bin = password_bin + delimiter + message_bin

    # Load the image
    image = cv2.imread(image_path)
    pixels = np.array(image)

    # Embed the binary data into the image using LSB method
    data_index = 0
    for row in range(pixels.shape[0]):
        for col in range(pixels.shape[1]):
            for channel in range(pixels.shape[2]):
                if data_index < len(data_bin):
                    pixel_bin = format(pixels[row, col, channel], '08b')
                    pixels[row, col, channel] = int(
                        pixel_bin[:-1] + data_bin[data_index], 2)
                    data_index += 1
                else:
                    break

    # Save the stego image
    stego_image = np.array(pixels, dtype=np.uint8)
    cv2.imwrite(output_path + 'stego_image.png', stego_image)


def decrypt(password, stego_path, output_path):
    output_path += '/'
    # Load the stego image
    stego = cv2.imread(stego_path, cv2.IMREAD_UNCHANGED)

    # Convert the password to binary
    password_bin = message_to_bin(password)

    # Extract the binary data from the stego image using LSB method
    data_bin = ''
    for row in range(stego.shape[0]):
        for col in range(stego.shape[1]):
            for channel in range(stego.shape[2]):
                data_bin += str(stego[row, col, channel] & 1)

    # Use the delimiter to separate password and message binary
    delimiter = '10101010'
    password_end_index = data_bin.find(delimiter)

    if password_end_index != -1:
        # Extract password and message binary
        password_bin_extracted = data_bin[:password_end_index]
        message_bin = data_bin[password_end_index + len(delimiter):]

        # Ensure that the length of the message binary is a multiple of 8
        message_bin = message_bin[:len(message_bin) - (len(message_bin) % 8)]

        # Verify the extracted password
        if password_bin_extracted == message_to_bin(password):
            # Convert the remaining binary data to characters
            text = bin_to_message(message_bin)
            # Save the extracted text to the output path
            output_text_path = os.path.join(output_path, 'decrypted_text.txt')
            with open(output_text_path, 'w', encoding='utf-8') as f:
                f.write(text)
            print("Decryption successful. Text saved to:", output_text_path)
        else:
            print("Incorrect password.")
    else:
        print("Delimiter not found in the image.")

# UI

In [ ]:
# layout of the GUI
layout = [
    [sg.Text('Encryption Section', font=(
        'Helvetica', 16), justification='center')],
    [sg.Text('Enter password for encryption:')],
    [sg.InputText(key='encrypt_password',)],
    [sg.Text('Select a text file for encryption:')],
    [sg.InputText(key='text_file'), sg.FileBrowse(
        file_types=(("Text Files", "*.txt"), ("All Files", "*.*")))],
    [sg.Text('Select an image for encryption:')],
    [sg.InputText(key='image_file'), sg.FileBrowse(file_types=(
        ("Image Files", "*.png;*.jpg;*.jpeg;*.gif"), ("All Files", "*.*")))],
    [sg.Text('Select an output directory for encrypted files:')],
    [sg.InputText(key='output_dir_en'), sg.FolderBrowse()],
    [sg.Button('Encrypt and Save'),],

    [sg.Text('Decryption Section', font=(
        'Helvetica', 16), justification='center')],
    [sg.Text('Enter password for decryption:')],
    [sg.InputText(key='password')],
    [sg.Text('Select a Image file for decryption:')],
    [sg.InputText(key='decrypt_image_file'), sg.FileBrowse(file_types=(
        ("Image Files", "*.png;*.jpg;*.jpeg;*.gif"), ("All Files", "*.*")))],
    [sg.Text('Select an output directory for encrypted files:')],
    [sg.InputText(key='output_dir_dec'), sg.FolderBrowse()],
    [sg.Button('Decrypt Image'), sg.Button('Exit')],
]

# Create the window
window = sg.Window('Encryption and Decryption App', layout, resizable=True)

while True:
    event, values = window.read()

    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    if event == 'Encrypt and Save':
        password = values['encrypt_password']
        text_file = values['text_file']
        image_file = values['image_file']
        stego_output_dir = values['output_dir_en']
        if not password or not text_file or not image_file or not stego_output_dir:
            sg.popup_error('Please Select Avalibal Options')
        else:
            encrypt(password=password, text_path=text_file,
                    image_path=image_file, output_path=stego_output_dir)
            sg.popup('image is encrypted')
        pass
    if event == 'Decrypt Image':
        decrypt_image_file = values['decrypt_image_file']
        password = values['password']
        decrypt_image_file_output_dir = values['output_dir_dec']
        if not decrypt_image_file or not password:
            sg.popup_error(
                'Please select a Image file and enter a password for decryption.')
        else:
            decrypt(password=password, stego_path=decrypt_image_file,
                    output_path=decrypt_image_file_output_dir)
            sg.popup('Decrypted image and text')

# Close the window
window.close()